# Filtering by custodial sentence length

In [ ]:
import pandas as pd

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.data.processing import filter_sentence_length

In [ ]:
df = filter_sentence_length.load_and_process_data()

In [ ]:
df.info()

In [ ]:
categories = df['sentence_len'].cat.categories
categories

In [ ]:
df

In [ ]:
df.groupby(['pfa', 'year'], as_index=False, observed=True)['freq'].sum()

In [ ]:
filter_sentence_length.load_and_process_data()